# Training Demo
Demonstrate how to use training utilities to fine-tune an LLM.

## Working Directory and Libraries

In [1]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/MIDS/compositional-reasoning-finetuning')

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00


In [ ]:
import os
import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from transformers import T5Tokenizer, TFT5ForConditionalGeneration

# 2WikiMultiHopQA

## Load and format data

In [ ]:
from data_loaders import load_FinetuningData
from training_utils import qa_split

In [ ]:
# Load data
data = load_FinetuningData(n_examples=5, split="dev", strategy="direct")
questions, answers = qa_split(data)

In [ ]:
print(questions)
print(answers)

['Facts:\nFact #1: Vaettiya Madichu Kattu  is a 1998 Tamil film written and directed by K. Bhagyaraj.\nFact #2: Krishnasaamy Bhagyaraj (born 7 January 1953) is an Indian director, actor, screenwriter, music director, producer and Politician active mainly in Tamil films.\n\nQuestion: What is the date of birth of the director of film Vaettiya Madichu Kattu?\nAnswer:', 'Facts:\nFact #1: She is the wife of Raila Odinga, the former Prime Minister of Kenya.\nFact #2: Son of the first Vice President of Kenya, Jaramogi Oginga Odinga, he draws a large chunk of his support from various regions in Kenya, most notably the Coastal Region and his native former Nyanza Province.\n\nQuestion: Who is the father-in-law of Ida Odinga?\nAnswer:', "Facts:\nFact #1: He was the eldest son of Bernhard I, Prince of Anhalt-Bernburg, by his wife Princess Sophie, daughter of King Abel of Denmark.\nFact #2: He was the second son of Henry I, Count of Anhalt (who was elevated to the rank of prince in 1218), by his wi

In [ ]:
# iterate over question answer pairs and print them
for question, answer in zip(questions, answers):
    print(question)
    print(answer)
    print()

Facts:
Fact #1: Vaettiya Madichu Kattu  is a 1998 Tamil film written and directed by K. Bhagyaraj.
Fact #2: Krishnasaamy Bhagyaraj (born 7 January 1953) is an Indian director, actor, screenwriter, music director, producer and Politician active mainly in Tamil films.

Question: What is the date of birth of the director of film Vaettiya Madichu Kattu?
Answer:
7 January 1953

Facts:
Fact #1: She is the wife of Raila Odinga, the former Prime Minister of Kenya.
Fact #2: Son of the first Vice President of Kenya, Jaramogi Oginga Odinga, he draws a large chunk of his support from various regions in Kenya, most notably the Coastal Region and his native former Nyanza Province.

Question: Who is the father-in-law of Ida Odinga?
Answer:
Jaramogi Oginga Odinga

Facts:
Fact #1: He was the eldest son of Bernhard I, Prince of Anhalt-Bernburg, by his wife Princess Sophie, daughter of King Abel of Denmark.
Fact #2: He was the second son of Henry I, Count of Anhalt (who was elevated to the rank of prince

In [ ]:
# Load self-ask data
data = load_FinetuningData(n_examples=5, split="dev", strategy="self_ask")
questions, answers = qa_split(data)

In [ ]:
print(questions[0])
print(answers[0])

Examples:
START
Question: Who is Afonso Of Portugal, Lord Of Portalegre's paternal grandmother?
First break the reasoning down into intermediate questions, then provide an answer.
Intermediate question: What is the father of Afonso of Portugal?
Intermediate answer: Afonso III of Portugal
Intermediate question: Who is the mother of Afonso III of Portugal?
Intermediate answer: Urraca of Castile
So the answer is Urraca of Castile.

END

START
Question: Are both directors of films Undertow (1930 film) and Boomerang (1992 film) from the same country?
First break the reasoning down into intermediate questions, then provide an answer.
Intermediate question: Who is the director of Boomerang (1992 film)?
Intermediate answer: Reginald Hudlin
Intermediate question: Who is the director of Undertow (1930 film)?
Intermediate answer: Harry A. Pollard
Intermediate question: What is the country of citizenship of Reginald Hudlin?
Intermediate answer: American
Intermediate question: What is the country o

## Demo of MultihopQADataGenerator

In [ ]:
# Load the pretrained tensorflow model

model_name = 't5-base'
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = TFT5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
train_file = 'data/FinetuningData/self_ask_train-answer_first=False-random_facts=False.json'
valid_file = 'data/FinetuningData/self_ask_dev-answer_first=False-random_facts=False.json'

In [ ]:
n_train_pairs = 154876
n_valid_pairs = 12576

In [ ]:
import json

f_train = open(train_file)
f_valid = open(valid_file)

js_train = json.load(f_train)
js_valid = json.load(f_valid)

# Close JSON file
f_train.close()
f_valid.close()

n_train_pairs = len(js_train) #154876
n_valid_pairs = len(js_valid) #12576

del js_train
del js_valid

In [ ]:
# Create the data generators for train and validation data, tensorflow version
from training_utils import MultihopQADataGenerator

max_length = 32
max_length = 512
batch_size = 16

train_data_generator = MultihopQADataGenerator(
    tokenizer=t5_tokenizer,
    model=t5_model,
    n_examples=n_train_pairs,
    data_filename=train_file,
    max_length=max_length,
    batch_size=batch_size
)

valid_data_generator = MultihopQADataGenerator(
    tokenizer=t5_tokenizer,
    model=t5_model,
    n_examples=n_valid_pairs,
    data_filename=valid_file,
    max_length=max_length,
    batch_size=batch_size
)

In [ ]:
def build_t5_training_wrapper_model(t5_model, max_length):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_mask')
    decoder_input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='labels')

    t5_logits = t5_model(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)[0]

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, decoder_input_ids],
                                  outputs=[t5_logits])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

In [ ]:
model_wrapper = build_t5_training_wrapper_model(t5_model, max_length)

In [ ]:
# As in the first notebook, we should add a model checkpoint callback to save
# the trained model weights after each epoch. Edit the filepath to where
# you want to save the weights in your own Drive

# checkpoint_dir = 'drive/MyDrive/266/data/self_ask/model_checkpoints/'
# checkpoint_dir = 'drive/MyDrive/projects/compositional-reasoning-finetuning/checkpoints/t5-base-self-ask/' # richard
checkpoint_filepath = os.getcwd() + '/checkpoints/{model_name}-self-ask/t5_direct_weights.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True)

In [ ]:
# Now call .fit on the model_wrapper, passing in the data generators and the
# model checkpoint callback

model_wrapper.fit(train_data_generator,
                  validation_data=valid_data_generator,
                  epochs=1,
                  callbacks=[model_checkpoint_callback])

ResourceExhaustedError: ignored

## Standardized function call for all models

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
import os
import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
os.chdir('/content/drive/MyDrive/MIDS/compositional-reasoning-finetuning')

In [ ]:
from training_utils import finetune_self_ask
import json

# model_name
model_name = 't5-base'

# traing and validation file path
#train_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_train.json'
#valid_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_dev.json'
train_file = 'data/finetuning/self_ask_train.json'
valid_file = 'data/finetuning/self_ask_dev.json'

# path and file name for checkpoint
checkpoint_dir = 'model_checkpoints/'
checkpoint_filepath = checkpoint_dir + model_name + '_self_ask_weights.{epoch:02d}-{batch:00005d}.hdf5'

# hyper parameters
max_length = 600
batch_size = 16
epochs = 2

model_wrapper = finetune_self_ask(model_name, train_file, valid_file, checkpoint_filepath, max_length, batch_size, epochs)

## Function call to filter json by token size

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd drive/MyDrive/266/compositional_reasoning

In [ ]:
train_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_train.json'
valid_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_dev.json'
token_size = 470

In [ ]:
filter_token_size(train_file, valid_file, token_size)

## Load previously saved checkpoint

In [ ]:
from training_utils import finetune_self_ask
import json

# model_name
model_name = 't5-small'

# traing and validation file path
train_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_train_300.json'
valid_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_dev_300.json'
#train_file = 'data/finetuning/self_ask_train_480.json'
#valid_file = 'data/finetuning/self_ask_dev_480.json'

previous_checkpoint = "./model_checkpoints/t5-small_v100_self_ask_300_weights.02-00704.hdf5"

# path and file name for checkpoint
checkpoint_dir = '/content/drive/MyDrive/266/compositional_reasoning/model_checkpoints/'
checkpoint_filepath = checkpoint_dir + model_name + '_v100_self_ask_300_weights.{epoch:02d}-{batch:00005d}.hdf5'

# hyper parameters
max_length = 300
batch_size = 32
epochs = 2

model_wrapper = finetune_self_ask(model_name, train_file, valid_file, checkpoint_filepath, max_length, batch_size, epochs, previous_checkpoint)

# HotPotQA

## Load and format data

In [ ]:
from data_loaders import load_FinetuningData
from training_utils import qa_split, tokenize

In [ ]:
# Load data
data = load_FinetuningData(n_examples=5, split="dev", strategy="direct")
questions, answers = qa_split(data)

In [ ]:
print(questions)
print(answers)

In [ ]:
# iterate over question answer pairs and print them
for question, answer in zip(questions, answers):
    print(question)
    print(answer)
    print()

In [ ]:
# Load self-ask data
data = load_FinetuningData(n_examples=5, split="dev", strategy="self_ask")
questions, answers = qa_split(data)

In [ ]:
print(questions[0])
print(answers[0])

## Demo of MultihopQADataGenerator

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
import os
import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the pretrained tensorflow model

model_name = 't5-base'
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = TFT5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# richard
train_file = 'data/FinetuningData/self_ask_train.json'
valid_file = 'data/FinetuningData/self_ask_dev.json'

%cd drive/MyDrive/projects/compositional-reasoning-finetuning

In [ ]:
n_train_pairs = 154876
n_valid_pairs = 12576

In [ ]:
train_file = 'drive/MyDrive/266/data/self_ask_train.json'
valid_file = 'drive/MyDrive/266/data/self_ask_dev.json'

In [ ]:
import json

f_train = open('drive/MyDrive/266/data/self_ask_train.json')
f_valid = open('drive/MyDrive/266/data/self_ask_dev.json')

js_train = json.load(f_train)
js_valid = json.load(f_valid)

# Close JSON file
f_train.close()
f_valid.close()

n_train_pairs = len(js_train) #154876
n_valid_pairs = len(js_valid) #12576

del js_train
del js_valid

In [ ]:
# Create the data generators for train and validation data, tensorflow version
from training_utils import MultihopQADataGenerator

max_length = 32
max_length = 512
batch_size = 16

train_data_generator = MultihopQADataGenerator(
    tokenizer=t5_tokenizer,
    model=t5_model,
    n_examples=n_train_pairs,
    data_filename=train_file,
    max_length=max_length,
    batch_size=batch_size
)

valid_data_generator = MultihopQADataGenerator(
    tokenizer=t5_tokenizer,
    model=t5_model,
    n_examples=n_valid_pairs,
    data_filename=valid_file,
    max_length=max_length,
    batch_size=batch_size
)

In [ ]:
def build_t5_training_wrapper_model(t5_model, max_length):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_mask')
    decoder_input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='labels')

    t5_logits = t5_model(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)[0]

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, decoder_input_ids],
                                  outputs=[t5_logits])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

In [ ]:
model_wrapper = build_t5_training_wrapper_model(t5_model, max_length)

In [ ]:
# As in the first notebook, we should add a model checkpoint callback to save
# the trained model weights after each epoch. Edit the filepath to where
# you want to save the weights in your own Drive

# checkpoint_dir = 'drive/MyDrive/266/data/self_ask/model_checkpoints/'
checkpoint_dir = 'drive/MyDrive/projects/compositional-reasoning-finetuning/checkpoints/t5-base-self-ask/' # richard
checkpoint_filepath = checkpoint_dir + 't5_direct_weights.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True)

In [ ]:
# Now call .fit on the model_wrapper, passing in the data generators and the
# model checkpoint callback

model_wrapper.fit(train_data_generator,
                  validation_data=valid_data_generator,
                  epochs=1,
                  callbacks=[model_checkpoint_callback])

## Standardized function call for all models

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
import os
import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd drive/MyDrive/266/compositional_reasoning

In [ ]:
from training_utils import finetune_self_ask
import json

# model_name
model_name = 't5-base'

# traing and validation file path
#train_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_train.json'
#valid_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_dev.json'
train_file = 'data/finetuning/self_ask_train.json'
valid_file = 'data/finetuning/self_ask_dev.json'

# path and file name for checkpoint
checkpoint_dir = 'model_checkpoints/'
checkpoint_filepath = checkpoint_dir + model_name + '_self_ask_weights.{epoch:02d}-{batch:00005d}.hdf5'

# hyper parameters
max_length = 600
batch_size = 16
epochs = 2

model_wrapper = finetune_self_ask(model_name, train_file, valid_file, checkpoint_filepath, max_length, batch_size, epochs)

## Function call to filter json by token size

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd drive/MyDrive/266/compositional_reasoning

In [ ]:
train_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_train.json'
valid_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_dev.json'
token_size = 470

In [ ]:
filter_token_size(train_file, valid_file, token_size)

## Load previously saved checkpoint

In [ ]:
from training_utils import finetune_self_ask
import json

# model_name
model_name = 't5-small'

# traing and validation file path
train_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_train_300.json'
valid_file = '/content/drive/MyDrive/266/compositional_reasoning/data/finetuning/self_ask_dev_300.json'
#train_file = 'data/finetuning/self_ask_train_480.json'
#valid_file = 'data/finetuning/self_ask_dev_480.json'

previous_checkpoint = "./model_checkpoints/t5-small_v100_self_ask_300_weights.02-00704.hdf5"

# path and file name for checkpoint
checkpoint_dir = '/content/drive/MyDrive/266/compositional_reasoning/model_checkpoints/'
checkpoint_filepath = checkpoint_dir + model_name + '_v100_self_ask_300_weights.{epoch:02d}-{batch:00005d}.hdf5'

# hyper parameters
max_length = 300
batch_size = 32
epochs = 2

model_wrapper = finetune_self_ask(model_name, train_file, valid_file, checkpoint_filepath, max_length, batch_size, epochs, previous_checkpoint)